In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
from numpy.linalg import norm
import math 
from scipy import spatial
import copy

In [ ]:
#Đọc file training
training_df = pd.read_excel(r"/content/gdrive/MyDrive/KLTN/20210421/Data/training.xlsx")
training_df = training_df.rename(columns={'Column1':'User', 'Column2':'Item', 'Column3':'Rating'}, errors="raise")
del training_df["Column4"]

In [ ]:
#Hàm tính Jaccard sim giữa hai người dùng
#Input: file training, 2 người dùng
#Output: Gía trị tương đồng giữa hai người dùng
def cal_sim(TrainingDataFrame, User1, User2):

  #Lấy danh sách item của 2 người dùng mà có đánh giá là 1 
  User1_items = TrainingDataFrame.Item[TrainingDataFrame.User == User1][TrainingDataFrame.Rating == 1].to_list()
  User2_items = TrainingDataFrame.Item[TrainingDataFrame.User == User2][TrainingDataFrame.Rating == 1].to_list()

  #Giao của 2 người dùng
  intersection = [value for value in User1_items if value in User2_items]

  #Chặn lại trong trường hợp không có item chung hay là mẫu bằng 0
  if len(intersection) == 0:
    return 0
  if (len(User1_items) + len(User2_items) - len(intersection)) == 0:
    return 0
  
  #Gía trị tương đồng Jaccard
  sim = len(intersection) / (len(User1_items) + len(User2_items) - len(intersection)) 
  
  return sim 

In [ ]:
#Hàm trả về các giá trị tương đồng của người dùng được chọn với những người dùng còn lại
#Input: 
#TrainingDataFrame - bộ dữ liệu traning dưới dạng dataframe, User - id người dùng mục tiêu
#Output:
#Trả về một Dataframe chứa độ tương đồng của người dùng mục tiêu với những người dùng khác 
def sim_df(TrainingDataFrame, User):
  User_col =[]
  Sim_col = []
  #Vòng lặp hết người dùng
  for i in TrainingDataFrame.User.drop_duplicates():
    if User != i:
      User_col.append(i)
      Sim_col.append(cal_sim(TrainingDataFrame, User, i))
  
  #Tạo dataframe 
  df = pd.DataFrame(data={'User':User_col, 'Sim':Sim_col})

  #Sắp xếp độ tương đồng
  df = df.sort_values(by=['Sim'], axis=0,ascending=False, ignore_index=True)

  return df

In [ ]:
#hàm trả về file excel
#Input:
#TrainingDataFrame - file training dưới dạng dataframe, path - vị trí lưu, first - người dùng đầu tiên cho vòng lặp, last - người dùng cuối cho vòng lặp 
#Output:
#Danh sách các file excel chứa độ tương đồng của người dùng
def to_excel(TrainingDataFrame, path, first, last):
    for i in range(first, last + 1):
        df = sim_df(TrainingDataFrame, i)
        print(i)
        address = r"{0}/{1}.xlsx".format(path, i)
        df.to_excel(address, index=False)

In [ ]:
to_excel(training_df, "/content/gdrive/MyDrive/KLTN/20210421/K-NN Jaccard Sim", 1, 6040)